# 地方競馬（NAR）初期データ分析 (EDA)

このノートブックでは、`NarDataLoader` を介してロードされた地方競馬（NAR）データの初期概要を確認します。
JRAでの知見をベースに、地方競馬特有のデータ傾向（競馬場の分布、距離、馬場状態など）を分析します。

In [ ]:
import sys
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

# プロジェクトのsrcディレクトリをパスに追加
src_path = os.path.abspath(os.path.join(os.getcwd(), '../../src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from nar.loader import NarDataLoader

%matplotlib inline
sns.set(font="IPAexGothic")

## 1. データのロード

`NarDataLoader` を使用して地方競馬データをロードします。

In [ ]:
loader = NarDataLoader()
# 初期の分析用に10万件程度をサンプルとして読み込みます
df = loader.load(limit=100000)
print(f"{len(df)} 件のデータをロードしました。")

In [ ]:
df.head()

## 2. データの基本情報

カラムの型や欠損値の状況を確認します。

In [ ]:
df.info()

In [ ]:
print("欠損値の数:")
print(df.isnull().sum())

## 3. 基本統計量と分布

数値データの分布を確認します。

In [ ]:
df[['odds', 'popularity', 'weight', 'distance', 'impost']].describe()

### 3.1 競馬場の分布

地方競馬は多くの競馬場が存在するため、その偏りを確認します。

In [ ]:
plt.figure(figsize=(15, 6))
sns.countplot(data=df, x='venue', order=df['venue'].value_counts().index)
plt.title("競馬場別のデータ件数")
plt.xlabel("競馬場コード")
plt.ylabel("件数")
plt.show()

### 3.2 距離と馬場状態

地方競馬はダートがメインですが、距離設定や天候による馬場状態の分布を確認します。

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

sns.histplot(df['distance'], bins=20, ax=axes[0])
axes[0].set_title("距離の分布")

sns.countplot(data=df, x='state', ax=axes[1])
axes[1].set_title("馬場状態の分布")

plt.show()

### 3.3 着順と人気の関係

人気と実際の着順（Rank）の相関を視覚化します。

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[df['rank'] <= 10], x='rank', y='popularity')
plt.title("着順と人気の関係")
plt.xlabel("着順")
plt.ylabel("人気")
plt.show()

## 4. 拡張分析：クラス・配当・人間系データ

拡張された `loader` を使用して、より詳細な分析を行います。

### 4.1 レースの「条件」別の分布

グレードコード `grade_code` の分布を確認します。データに空白が含まれる場合は除去します。

In [ ]:
if 'grade_code' in df.columns:
    # 空白文字を除去し、空文字列を '(空白)' などに置換して可視化しやすくする
    plot_df = df.copy()
    plot_df['grade_code'] = plot_df['grade_code'].astype(str).str.strip()
    plot_df.loc[plot_df['grade_code'] == '', 'grade_code'] = '(空白)'
    
    plt.figure(figsize=(12, 4))
    sns.countplot(data=plot_df, x='grade_code', order=sorted(plot_df['grade_code'].unique()))
    plt.title("レースグレード(nvd_ra.grade_code)の分布")
    plt.show()
else:
    print("grade_code カラムが見つかりません。")

### 4.2 払戻金データの分析 (ROIシミュレーションの基盤)

直近の払戻データを読み込んで、単勝の平均配当等を確認します。

In [ ]:
try:
    # 最新の日付付近の払戻データを取得してみる (サンプルとして特定の日付)
    sample_date = df['date'].max().strftime('%Y-%m-%d')
    payouts_df = loader.load_payouts(sample_date)
    print(f"{sample_date} の払戻レコード数: {len(payouts_df)}")
    
    if not payouts_df.empty:
        payouts_df.head()
        
        plt.figure(figsize=(10, 4))
        sns.histplot(payouts_df['payout_win_amount'].dropna(), bins=30, kde=True)
        plt.title(f"{sample_date} の単勝配当分布")
        plt.xlabel("配当金(円)")
        plt.show()
    else:
        print(f"{sample_date} の払戻データは見つかりませんでした。")
except Exception as e:
    print(f"払戻データの取得中にエラーが発生しました: {e}")

### 4.3 騎手・調教師マスタの読み込み

IDだけでなく、名前等の情報を取得できるマスタの状況を確認します。

In [ ]:
try:
    masters = loader.load_human_master()
    jockeys = masters['jockeys']
    trainers = masters['trainers']
    print(f"登録騎手ユーザー数: {len(jockeys)}")
    print(f"登録調教師ユーザー数: {len(trainers)}")
    if not jockeys.empty:
        print("\n--- 騎手データサンプル ---")
        display(jockeys.head())
except Exception as e:
    print(f"マスタデータの取得中にエラーが発生しました: {e}")

## 5. 今後の検証課題

このベース分析を踏まえ、以下の点について今後深掘りしていきます。
1. 競馬場ごとの特徴（大井、門別など）に特化した分析。
2. 騎手・調教師の地方競馬における影響度（勝率や回収率の集計）。
3. JRAから移籍してきた馬（いわゆる転入馬）の成績傾向。
4. ナイター開催や季節性の影響分析。